In [2]:
import scvelo as scv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scanpy as sc
import glob
import os

In [3]:
sample = 'day7'
results_file = '/home/linxy29/holab/iPSC/veloAE/' + sample + '_seurate.h5ad'  # the file that will store the analysis results

In [4]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.21.6 scipy==1.7.3 pandas==1.3.5 scikit-learn==1.0.2 statsmodels==0.13.2 pynndescent==0.5.7


In [5]:
# get meta-data information
path = "/home/linxy29/holab/iPSC/veloAE/"
cellID_obs = pd.read_csv(path + sample + '_cellID_obs.csv')
print(cellID_obs.head())
print(len(cellID_obs))
umap_cord = pd.read_csv(path + sample + '_cell_embeddings.csv')
print(umap_cord)
cell_clusters = pd.read_csv(path + sample + '_clusters.csv')
print(cell_clusters)

                    x
0  AAACCCAAGCCGATCC-1
1  AAACCCAAGTACAGCG-1
2  AAACCCAGTGCTGATT-1
3  AAACCCAGTTTGGCTA-1
4  AAACCCATCACTTGGA-1
9744
              Unnamed: 0    UMAP_1    UMAP_2
0     AAACCCAAGCCGATCC-1  0.153307 -7.623546
1     AAACCCAAGTACAGCG-1  5.897146  2.633477
2     AAACCCAGTGCTGATT-1 -5.536019 -6.625185
3     AAACCCAGTTTGGCTA-1 -7.391526 -0.720508
4     AAACCCATCACTTGGA-1 -2.355847 -5.528623
...                  ...       ...       ...
9739  TTTGTTGGTAGACGTG-1 -1.424601 -6.192325
9740  TTTGTTGGTGAGCGAT-1  6.826642  9.918284
9741  TTTGTTGGTTCTCTAT-1  3.751490  3.653710
9742  TTTGTTGTCACCTCAC-1 -3.714295  2.119375
9743  TTTGTTGTCGGATAAA-1 -5.586266 -0.573459

[9744 rows x 3 columns]
              Unnamed: 0                                      x
0     AAACCCAAGCCGATCC-1                          Neuroectoderm
1     AAACCCAAGTACAGCG-1                              Notochord
2     AAACCCAGTGCTGATT-1            Neuromesodermal progenitors
3     AAACCCAGTTTGGCTA-1            Primit

In [6]:
filenameL = []
os.chdir("/home/linxy29/holab/iPSC/veloAE/")
for file in glob.glob("*.loom"):
    filenameL.append(file)
print(filenameL)

['ThermoFiPSC_D7NT_sample_alignments_Z46BN.loom', 'ThermoFiPSC_D7NOTO_sample_alignments_BT23H.loom', 'ThermoFiPSC_D7NOTO_drug_sample_alignments_PUHUH.loom', '190PBMCiPSC_D7NT_sample_alignments_LNJIY.loom', '190PBMCiPSC_D7NOTO_sample_alignments_OV87U.loom', '190PBMCiPSC_D7NOTO_drug_sample_alignments_P1WVY.loom']


In [7]:
ldata =  scv.read("/home/linxy29/holab/iPSC/veloAE/" + filenameL[0], cache=True)
ldata.var_names_make_unique()

... reading from cache file cache/home-linxy29-holab-iPSC-veloAE-ThermoFiPSC_D7NT_sample_alignments_Z46BN.h5ad


In [8]:
for sample in filenameL[1:]:
    tempdata = scv.read("/home/linxy29/holab/iPSC/veloAE/" + sample, cache=True)
    tempdata.var_names_make_unique()
    ldata = ldata.concatenate(tempdata, join='outer', index_unique=None)
print(ldata)
obs_name_temp = ldata.obs_names.str.split(":")
ldata.obs_names = [item[1] + '-1' for item in obs_name_temp]
print(ldata.obs_names)

... reading from cache file cache/home-linxy29-holab-iPSC-veloAE-ThermoFiPSC_D7NOTO_sample_alignments_BT23H.h5ad
... reading from cache file cache/home-linxy29-holab-iPSC-veloAE-ThermoFiPSC_D7NOTO_drug_sample_alignments_PUHUH.h5ad
... reading from cache file cache/home-linxy29-holab-iPSC-veloAE-190PBMCiPSC_D7NT_sample_alignments_LNJIY.h5ad
... reading from cache file cache/home-linxy29-holab-iPSC-veloAE-190PBMCiPSC_D7NOTO_sample_alignments_OV87U.h5ad
... reading from cache file cache/home-linxy29-holab-iPSC-veloAE-190PBMCiPSC_D7NOTO_drug_sample_alignments_P1WVY.h5ad
AnnData object with n_obs × n_vars = 10656 × 61544
    obs: 'batch'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'
Index(['AAGGTAACAGGTTTAC-1', 'AACCTGACATGAAGCG-1', 'AAGGAATGTCGCGTTG-1',
       'AACCAACAGTCACGAG-1', 'AAGAACAGTATTCCTT-1', 'AAGACTCTCTTTGATC-1',
       'AAGCGTTTCATAGAGA-1', 'AACCAACTCGCCGATG-1', 'AAGCGAGGTCTTGTCC-1',
       'ACGATGTGTGGAG

## barcode

In [9]:
## change obs_names
print(ldata.obs_names)
print(np.unique(ldata.obs_names).size == len(ldata.obs_names))
print(len(ldata.obs_names))

Index(['AAGGTAACAGGTTTAC-1', 'AACCTGACATGAAGCG-1', 'AAGGAATGTCGCGTTG-1',
       'AACCAACAGTCACGAG-1', 'AAGAACAGTATTCCTT-1', 'AAGACTCTCTTTGATC-1',
       'AAGCGTTTCATAGAGA-1', 'AACCAACTCGCCGATG-1', 'AAGCGAGGTCTTGTCC-1',
       'ACGATGTGTGGAGAAA-1',
       ...
       'TTGGATGTCAACGTGT-1', 'TTAGGGTGTCAGTCGC-1', 'TTTCGATCAAGACGGT-1',
       'TTTGGTTAGTGAGCCA-1', 'TTTATGCTCAGGACGA-1', 'TTTGATCAGTGGCCTC-1',
       'TTTGTTGGTAGACGTG-1', 'TTTCGATTCCTGGGAC-1', 'TTTCGATAGTAGGATT-1',
       'TTTGGAGAGGTAACTA-1'],
      dtype='object', length=10656)
True
10656


In [10]:
filtered_barcode = np.intersect1d(cellID_obs['x'],ldata.obs_names)
len(filtered_barcode)

9246

In [11]:
filtered_ldata = ldata[filtered_barcode].copy()
print(filtered_ldata)

AnnData object with n_obs × n_vars = 9246 × 61544
    obs: 'batch'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'


### Add UMAP and annotation

In [12]:
ldata_index = pd.DataFrame(filtered_ldata.obs.index)
ldata_index.columns = ['CellID']
print(ldata_index)

                  CellID
0     AAACCCAAGCCGATCC-1
1     AAACCCAAGTACAGCG-1
2     AAACCCAGTGCTGATT-1
3     AAACCCAGTTTGGCTA-1
4     AAACCCATCACTTGGA-1
...                  ...
9241  TTTGTTGCATCTCAAG-1
9242  TTTGTTGGTAGACGTG-1
9243  TTTGTTGGTGAGCGAT-1
9244  TTTGTTGGTTCTCTAT-1
9245  TTTGTTGTCACCTCAC-1

[9246 rows x 1 columns]


In [13]:
umap_cord = umap_cord.rename(columns = {'Unnamed: 0':'CellID'})
umap_ordered = ldata_index.merge(umap_cord, on = "CellID")
print(umap_cord)

                  CellID    UMAP_1    UMAP_2
0     AAACCCAAGCCGATCC-1  0.153307 -7.623546
1     AAACCCAAGTACAGCG-1  5.897146  2.633477
2     AAACCCAGTGCTGATT-1 -5.536019 -6.625185
3     AAACCCAGTTTGGCTA-1 -7.391526 -0.720508
4     AAACCCATCACTTGGA-1 -2.355847 -5.528623
...                  ...       ...       ...
9739  TTTGTTGGTAGACGTG-1 -1.424601 -6.192325
9740  TTTGTTGGTGAGCGAT-1  6.826642  9.918284
9741  TTTGTTGGTTCTCTAT-1  3.751490  3.653710
9742  TTTGTTGTCACCTCAC-1 -3.714295  2.119375
9743  TTTGTTGTCGGATAAA-1 -5.586266 -0.573459

[9744 rows x 3 columns]


In [14]:
umap_ordered = umap_ordered.iloc[:,1:]
filtered_ldata.obsm['X_umap'] = umap_ordered.values

In [15]:
cell_clusters = cell_clusters.rename(columns = {'Unnamed: 0':'CellID'})
cell_clusters = ldata_index.merge(cell_clusters, on = "CellID")
print(cell_clusters)

                  CellID                                      x
0     AAACCCAAGCCGATCC-1                          Neuroectoderm
1     AAACCCAAGTACAGCG-1                              Notochord
2     AAACCCAGTGCTGATT-1            Neuromesodermal progenitors
3     AAACCCAGTTTGGCTA-1            Primitive streak (cluster4)
4     AAACCCATCACTTGGA-1            Neuromesodermal progenitors
...                  ...                                    ...
9241  TTTGTTGCATCTCAAG-1            Primitive streak (cluster4)
9242  TTTGTTGGTAGACGTG-1                          Neuroectoderm
9243  TTTGTTGGTGAGCGAT-1                       Cardiac mesoderm
9244  TTTGTTGGTTCTCTAT-1  Lateral/paraxial mesoderm progenitors
9245  TTTGTTGTCACCTCAC-1            Primitive streak (cluster3)

[9246 rows x 2 columns]


In [16]:
cell_clusters = cell_clusters.iloc[:,1:]
filtered_ldata.obs['seurat_clusters'] = cell_clusters.values
print(filtered_ldata)

AnnData object with n_obs × n_vars = 9246 × 61544
    obs: 'batch', 'seurat_clusters'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    obsm: 'X_umap'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'


In [17]:
filtered_ldata.write(results_file)